In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [ ]:
from __future__ import division 
import math
from scipy.optimize import newton
import numpy as np
import json
import requests
import pandas as pd

In [ ]:
%load_ext line_profiler

In [ ]:
headers = {
    'Accept':'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'en-US,en;q=0.8,ja;q=0.6,zh-CN;q=0.4',
    'Cache-Control':'no-cache',
    'Connection':'keep-alive',
    'Content-Length':'2',
    'Content-Type':'application/json',
    'Host':'gs.amac.org.cn',
    'Origin':'http://gs.amac.org.cn',
    'Pragma':'no-cache',
    'Referer':'http://gs.amac.org.cn/amac-infodisc/res/pof/fund/index.html',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest',
}
page = 0
size = 20

finalInfo = pd.DataFrame()
for page in range(2):
    
    url = 'http://huidu.tradeapi.yunkuanke.com/front/query/queryPositionList.do'
    r = requests.post(url % (page, size), headers=headers, data='{}')
    data = json.dumps(json.loads(r.content)['content'])
    datadf = pd.read_json(data)
    finalInfo = pd.concat([finalInfo,datadf], axis=0)
finalInfo.reset_index(drop=True, inplace=True)
# finalInfo.to_csv(savepath+'/'+ filename,header=True)
# finalInfo['trueURL'] = finalInfo['managerUrl'].apply(lambda x : x.split('/')[-1])  # customize

In [10]:
def AmerImpliedVol2(opt_price, price, forwards, strike, time,DFs,callPutInd=1.,n_steps=200,scale=1.):
    time_step=float(time/n_steps)
    time_sqrt=np.sqrt(time_step)

    forwards_previous=[price]
    forwards_previous.extend(forwards[:-1])
    exp_drift=np.array(forwards)/np.array(forwards_previous)

    step_DFs=np.array(DFs[1:])/np.array(DFs[:-1])

    tree_template=vertical+horizontal

    #vol=0.25
    def step_price(vol):
        power_vol=np.exp(vol*time_sqrt)
        down=1/power_vol
        p_up=(exp_drift-down)/(power_vol-down)
        p_down=1.-p_up

        horizontal=np.ones(shape=(1,n_steps+1))*power_vol
        horizontal[0,0]=1.
        horizontal=np.cumprod(horizontal,axis=1)

        vertical=np.ones(shape=(n_steps+1,1))/(power_vol*power_vol)
        vertical[0,0]=1.
        vertical=np.cumprod(vertical,axis=0)

        tree=price*(vertical*horizontal)

        cp=callPutInd
        def payoff(values):
            return np.maximum(cp*(values-strike),0.)

        payoffs=payoff(tree)
        del tree
        current=payoffs[:,n_steps]

        for i in xrange(n_steps,0,-1):
            after_step=step_DFs[i-1]*(p_up[i-1]*current[:i]+p_down[i-1]*current[1:])
            current=np.maximum(after_step,payoffs[:i,i-1])
            #print current
            #current[:-1]=after_step

        return scale*current[0]-opt_price

    seed=0.3

    return newton(step_price,seed,tol=0.5e-04)

In [ ]:
#---------------------------------------------------- Tree to price American Option and cal IV and Greeks---------------------#

In [264]:
def get_price_by_binomial_tree(vol,s,k,t,option_type =1,rf=0.02):
        """
        This function will make the same calculation but by Binomial Tree
        """
        n=100
        deltaT=t/n
        u = math.exp(vol*math.sqrt(deltaT))
        d=1.0/u
        rf = 0.02
        # Initialize our f_{i,j} tree with zeros
        fs = [[0.0 for j in xrange(i+1)] for i in xrange(n+1)]
        a = math.exp(rf*deltaT)   # math.exp is faster than  np.exp
        p = (a-d)/(u-d)
        oneMinusP = 1.0-p 
        # Compute the leaves, f_{N,j}
        if option_type == 1 :
            for j in xrange(i+1):
                fs[n][j]=max(s * u**j * d**(n-j) - k, 0.0)
        elif option_type == -1:
             for j in xrange(i+1):
                fs[n][j]=max(k - s * u**j * d**(n-j), 0.0)
        for i in xrange(n-1, -1, -1):
            for j in xrange(i+1):
                fs[i][j]=math.exp(-rf * deltaT) * (p * fs[i+1][j+1] +
                                                        oneMinusP * fs[i+1][j])
 
        return fs[0][0]

In [248]:
def newton_target_func(vol,optprice,*args):
    return get_price_by_binomial_tree(vol,*args) - optprice

In [267]:
def get_IV_by_tree(optprice,s,k,t,option_type,init_guess = 0.1):
    return newton(newton_target_func,init_guess,args=(optprice,s,k,t,option_type),tol = 0.5e-05)

In [ ]:
#------------------------------------   END   --------------------------------------------#

In [274]:
# test
vol0 = 0.3
s0 = 8
k0=9.2
t0=1
call = 1
put = -1

opt_price_AM = get_price_by_binomial_tree(vol0,s0,k0,t0,put)
opt_price_AM

1.6040513508816128

In [276]:
IV = get_IV_by_tree(opt_price_AM ,s0,k0,t0,put)
IV

0.29999999999373594

In [207]:
# greeks 
#delta
s1 = s0/100 +s0
delta =  (get_price_by_binomial_tree(vol0,s1,k0,t0,rf=0.02) - opt_price_AM) / (s1- s0)
delta

0.30878469270429865

In [208]:
# vega
vol1 = vol0/100 + vol0
vega = (get_price_by_binomial_tree(vol1,s0,k0,t0,rf=0.02) - opt_price_AM) / (vol1- vol0)
vega

2.820414325472695

In [212]:
#gamma
s2 = s1/100 +s1
delta2 = (get_price_by_binomial_tree(vol0,s2,k0,t0,rf=0.02) - get_price_by_binomial_tree(vol0,s1,k0,t0,rf=0.02)) / (s2- s1)
gamma = (delta2 - delta) / (s1 - s0) 
gamma

0.012887886776404693

In [193]:
%lprun -f get_IV_by_tree get_IV_by_tree(opt_price_AM ,s0,k0,t0)

In [130]:
def power(args):
    x,y = args
    return x**y
power((2,2))

4